In [5]:
import json
from ollama import Client

In [7]:
# Crea el cliente Ollama (por defecto corre en localhost:11434)
client = Client()

def generar_historias_llama(json_input_file, json_output_file, prompt_base):
    with open(json_input_file, "r", encoding="utf-8") as f:
        issues = json.load(f)

    for issue in issues:
        title = issue.get("title", "")
        body = issue.get("body", "")
        state = issue.get("state", "")
        author = issue.get("user", "")
        created = issue.get("created_at", "")
        issue_number = issue.get("number", "")

        context_lines = []
        context_lines.append(f"Issue #{issue_number} reportado por {author} (creado el {created}):\n")
        context_lines.append(f"Título: {title}\n")
        context_lines.append(f"Descripción:\n{body}\n")
        context_lines.append(f"Estado: {state}\n")
        context_lines.append("Comentarios:")

        comments = issue.get("comments", [])
        if comments:
            for comment in comments:
                c_author = comment.get("user", "")
                c_date = comment.get("created_at", "")
                c_body = comment.get("body", "").strip()
                context_lines.append(f"- {c_author} ({c_date}): {c_body}")
        else:
            context_lines.append("- No hay comentarios.")

        full_prompt = "\n".join(context_lines) + "\n\n" + prompt_base

        print(f"Generando historia de usuario para issue #{issue_number}...")

        #llamar al modelo LLM
        completion = client.chat(
            #model="llama3",  # aca va modelo que descargue con `ollama pull llama3` es el mejor 5gb de ram y requiere grafica
            model="tinyllama", # modelo liviano `ollama pull tinyllama` es el mejor 1.5gb de ram y anda en cpu
            messages=[
                {
                    "role": "system",
                    "content": "Eres un experto en redacción de historias de usuario."
                },
                {
                    "role": "user",
                    "content": full_prompt
                }
            ],
            options={
                "temperature": 0.2
            }
        )

        respuesta = completion["message"]["content"].strip()
        issue["historia_usuario"] = respuesta

    with open(json_output_file, "w", encoding="utf-8") as f:
        json.dump(issues, f, indent=2, ensure_ascii=False)

    print(f"Historias generadas y guardadas en '{json_output_file}'.")


In [ ]:
# Ejemplo de uso",
prompt_base = "Por favor, genera una historia de usuario detallada basada en este issue."
generar_historias_llama("issuesComments.json", "issues_con_historias.json", prompt_base)